# Tarefa 03

- Leia os enunciados com atenção
- Saiba que pode haver mais de uma resposta correta
- Insira novas células de código sempre que achar necessário
- Em caso de dúvidas, procure os Tutores
- Divirta-se :)

In [68]:
import pandas as pd
import requests

####  1) Lendo de APIs
Vimos em aula como carregar dados públicos do governo através de um API (*Application Programming Interface*). No exemplo de aula, baixamos os dados de pedidos de verificação de limites (PVL) realizados por estados, e selecionamos apenas aqueles referentes ao estado de São Paulo.

1. Repita os mesmos passos feitos em aula, mas selecione os PVLs realizados por municípios no estado do Rio de Janeiro.
2. Quais são os três *status* das solicitações mais frequentes na base? Quais são suas frequências?
3. Construa uma nova variável que contenha o ano do **status**. Observe que ```data_status``` vem como tipo *object* no **DataFrame**. Dica: você pode usar o método ```.str``` para transformar o tipo da variável em string, em seguida um método como [**slice()**](https://pandas.pydata.org/docs/reference/api/pandas.Series.str.slice.html) ou [**split()**](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.str.split.html).
4. Indique a frequência de cada ano do campo construído no item (3).

In [70]:
# 1) Seu código aqui
url = "https://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl"

In [71]:
# 2) Seu código aqui
r = requests.get(url)
r.status_code

200

In [72]:
# 3) Seu código aqui
data_json = r.json()
print(data_json['items'])

[{'id_pleito': 17823, 'tipo_interessado': 'Município', 'interessado': 'Boa Esperança', 'cod_ibge': 4103008, 'uf': 'PR', 'num_pvl': None, 'status': 'Deferido', 'num_processo': '19406.000575/2007-72', 'data_protocolo': '2007-12-26T00:00:00Z', 'tipo_operacao': 'Operação contratual interna', 'finalidade': 'Aquisição de terrenos, imóveis e/ou construção de instalações', 'tipo_credor': 'Instituição Financeira Nacional', 'credor': 'Agência de Fomento do Paraná S/A', 'moeda': 'Real', 'valor': 400000, 'pvl_assoc_divida': 1, 'pvl_contradado_credor': 0, 'data_status': '04/01/2008'}, {'id_pleito': 17834, 'tipo_interessado': 'Município', 'interessado': 'Verê', 'cod_ibge': 4128609, 'uf': 'PR', 'num_pvl': None, 'status': 'Deferido', 'num_processo': '19406.000579/2006-70', 'data_protocolo': '2006-09-06T00:00:00Z', 'tipo_operacao': 'Operação contratual interna', 'finalidade': 'Aquisição de máquinas, equipamentos e veículos', 'tipo_credor': 'Instituição Financeira Nacional', 'credor': 'Agência de Foment

In [73]:
df_rj = pd.DataFrame(data_json['items'])
df_rj['uf'].value_counts()

uf
MG    933
PR    911
RS    737
SP    632
SC    569
BA    219
MT     96
CE     95
GO     93
RJ     78
PE     72
MS     66
ES     61
PI     57
TO     54
PA     48
MA     43
RN     40
PB     37
RO     34
AL     30
SE     29
AC     21
AM     21
RR     10
DF     10
AP      4
Name: count, dtype: int64

In [74]:
status = df_rj.status.value_counts()
print('Os três status das solicitações mais frequentes são:\n')

for i in range(3):

   print(f'{i+1}º {status.index[i]}: {status[i]}')

Os três status das solicitações mais frequentes são:

1º Deferido: 2633
2º Deferido (PVL-IF): 1009
3º Arquivado: 449


/var/folders/k8/3l98n8ks1x735y8sgy33qnzm0000gn/T/ipykernel_20196/1767877636.py:6: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print(f'{i+1}º {status.index[i]}: {status[i]}')


In [75]:
df_rj['ano'] = df_rj['data_status'].str.slice(0,None)

df_rj['ano']

0       04/01/2008
1       19/09/2006
2       05/12/2011
3       13/01/2014
4       18/02/2011
           ...    
4995    29/07/2010
4996    12/11/2003
4997    22/05/2006
4998    15/12/2003
4999    20/11/2003
Name: ano, Length: 5000, dtype: object

In [76]:
df_rj['ano'].value_counts().sort_index(ascending=True)

ano
01/02/2008    2
01/02/2011    1
01/02/2024    2
01/03/2006    1
01/03/2010    2
             ..
31/10/2017    1
31/10/2019    5
31/10/2023    3
31/12/2014    1
31/12/2021    2
Name: count, Length: 2490, dtype: int64

####  2) Melhorando a interação com o API
Observe dois URLs de consultas diferentes, por exemplo o URL utilizado em aula, e o URL feito no exercício anterior. Compare-os e observe as diferenças.

1. Faça uma função em Python que recebe como argumento o UF da consulta e o tipo de interessado (```'Estado'```ou ```Município```), e que devolve os dados da consulta no formato *DataFrame*.
2. Quantas solicitações para o Estado podem ser consultadas para Minas Gerais com *status* em 'Arquivado por decurso de prazo' estão registradas?
3. Qual é o município da Bahia com mais solicitações deferidas?
4. Salve um arquivo .csv com os dados de solicitações da Bahia, com interessado = 'Estado'

In [99]:
#1) Seu código aqui
def filtro_uf_tipo(uf: str, tipo_interessado: str) :
    response = requests.get('https://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl')
    data = response.json()
    df = pd.DataFrame(data['items'])

    filtro = (df['uf']== uf) & (df['tipo_interessado'] == tipo_interessado)
    df_filtrado = df[filtro]

    return df_filtrado

In [119]:
# 2) Seu código aqui
uf = 'MG'
tipo_interessado = 'Município'

df_filtrado = filtro_uf_tipo(uf, tipo_interessado)
df_arquivado = df_filtrado[df_filtrado['status'] == 'Arquivado por decurso de prazo']
df_arquivado.head()
status = df_arquivado.status.value_counts()
print('O status das solicitações mais frequentes é:\n')

for i in range(1):

   print(f'{i+1}º {status.index[i]}: {status[i]}')


O status das solicitações mais frequentes é:

1º Arquivado por decurso de prazo: 62


/var/folders/k8/3l98n8ks1x735y8sgy33qnzm0000gn/T/ipykernel_20196/1209293331.py:13: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print(f'{i+1}º {status.index[i]}: {status[i]}')


In [117]:
# 3) Seu código aqui
uf = 'BA'
tipo_interessado = 'Município'

df_filtrado = filtro_uf_tipo(uf, tipo_interessado)
df_arquivado.head()
status = df_arquivado.status.value_counts()
print('O status das solicitações mais frequentes é:\n')

for i in range(1):

   print(f'{i+1}º {status.index[i]}: {status[i]}')

O status das solicitações mais frequentes é:

1º Arquivado por decurso de prazo: 31


/var/folders/k8/3l98n8ks1x735y8sgy33qnzm0000gn/T/ipykernel_20196/325295252.py:12: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print(f'{i+1}º {status.index[i]}: {status[i]}')


In [113]:
# 4) Seu código aqui
uf = 'BA'
tipo_interessado = 'Estado'

df_filtrado = filtro_uf_tipo(uf, tipo_interessado)
df_arquivado = df_filtrado[df_filtrado['status'] == 'Arquivado por decurso de prazo']
df_arquivado.head()
df_arquivado.to_csv('./bahiaEstado.csv', index= False, sep=';', decimal=',')